## 

# <center> PostgreSQL

___

## Reference

https://www.postgresqltutorial.com/postgresql-python/

https://www.youtube.com/watch?v=L2UO_slnDbk

https://www.tutorialspoint.com/python_data_access

___

In [2]:
# Imports
import psycopg2
import pandas as pd

___

## Dataset

In [3]:
# Load data
df = pd.read_table("data/mtcars.txt", encoding = "UTF-8", sep = ' ')

In [4]:
# Subset
df = df[['mpg', 'cyl', 'disp', 'hp']].reset_index().rename(columns={'index':'name'})

In [7]:
# View data
df.head()

,name,mpg,cyl,disp,hp
0,Mazda RX4,21.0,6,160.0,110
1,Mazda RX4 Wag,21.0,6,160.0,110
2,Datsun 710,22.8,4,108.0,93
3,Hornet 4 Drive,21.4,6,258.0,110
4,Hornet Sportabout,18.7,8,360.0,175


In [9]:
columns = ', '.join(df.keys())
columns

'name, mpg, cyl, disp, hp'

___

## Querying tables and executing queries

In [7]:
# Connector
def _connector(db, host, user, password, port):
    
    con = psycopg2.connect(
        database=db,
        host=host,
        user=user,
        password=password,
        port=port)
    
    return con

In [21]:
# Query tables
def consult_db():

    # Create conecction
    con = _connector(db='postgres', host="localhost", user="postgres", password="masterkey", port=5432)
    con.autocommit = True
    
    # Create cursor
    cur = con.cursor()

    # Execute query
    cur.execute("SELECT datname FROM pg_database;")

    # Print tabels
    db_list=[]
    for x in cur:
        db_list.append(x[0])
    print(db_list)
        
    # Close connection
    cur.close()
    con.close()

In [22]:
# Query
consult_db()

['postgres', 'template1', 'template0', 'PUC1', 'mt_cars', 'estoque_clientes_gelo']


In [24]:
# Execute queries
def execute_query(database, query):
    
    # Create conecction
    con = _connector(db='mt_cars', host="localhost", user="postgres", password="masterkey", port=5432)
    con.autocommit = True
    
    # Create cursor
    cur = con.cursor()
    
    # Execute query
    cur.execute(query)
    
    # Close connection
    cur.close()
    con.close()

___

## Create base

In [26]:
# Create data base
def create_db(query, db_name):
    
    # Connection
    con = _connector(db='mt_cars', host="localhost", user="postgres", password="masterkey", port=5432)
    con.autocommit = True
    
    # Create cursor
    cur = con.cursor()
    
    # Check if db already exists
    cur.execute("SELECT datname FROM pg_database;")

    db_list=[]
    for x in cur:
        db_list.append(x[0])
    
    # Return db_name if already exists else create db
    if db_name in db_list:
        print(str(db_name) + ' already exists')
    else:      
        cur.execute(query)
        
    # Close connection
    cur.close()
    con.close()

In [27]:
# Create
db_name = 'mt_cars'
query = f"CREATE DATABASE {db_name};"
create_db(query, db_name)

mt_cars already exists


## Create table

In [28]:
# Create table
def create_table(database, query):
    
    # Connection
    con = _connector(db='mt_cars', host="localhost", user="postgres", password="masterkey", port=5432)
    con.autocommit = True
    
    # Create cursor
    cur = con.cursor()

    # Execute query
    cur.execute(query)

    # Close connection
    cur.close()
    con.close()

In [29]:
# Create
database = 'mtcars'
query = 'CREATE TABLE IF NOT EXISTS MTCars(name TEXT NOT NULL, mpg TEXT NOT NULL, cyl TEXT NOT NULL, disp TEXT NOT NULL, hp TEXT)'

# Execute
create_table(database, query)

## Insert data

In [30]:
# Insert data
def insert_data(database, query):
    
    # Connection
    con = _connector(db='mt_cars', host="localhost", user="postgres", password="masterkey", port=5432)
    con.autocommit = True
    
    # Insert data
    values = []
    for i in range(0, len(df.index)):
        values.append(tuple(df.iloc[i:i+1].sum().astype(str))) # str??
    
    # Create cursor
    cur = con.cursor()

    # Execute query
    cur.executemany(query, values)
    
    # Close connection
    cur.close()
    con.close()

In [31]:
# Insert
database = 'mtcars'
query = """INSERT INTO mtcars (name, mpg, cyl, disp, hp) VALUES (%s, %s, %s, %s, %s)"""

insert_data(database, query)

## Query data

In [32]:
# Read data
def read_data(database, query):
    
    # Connection
    con = _connector(db='mt_cars', host="localhost", user="postgres", password="masterkey", port=5432)
    con.autocommit = True
    
    # Create cursor
    cur = con.cursor()

    # Execute query
    cur.execute(query)
      
    # Return Pandas Dataframe    
    df_back = cur.fetchall()
    col_names = [i[0] for i in cur.description]
    df_back = pd.DataFrame(df_back, columns = [col_names])
    
    return df_back
    
    # Close connection
    cur.close()
    con.close()

In [33]:
# Query
database = 'mtcars'
query = 'select * from MTCars'

read_data(database, query)

,name,mpg,cyl,disp,hp
0,Mazda RX4,21.0,6,160.0,110
1,Mazda RX4 Wag,21.0,6,160.0,110
2,Datsun 710,22.8,4,108.0,93
3,Hornet 4 Drive,21.4,6,258.0,110
4,Hornet Sportabout,18.7,8,360.0,175
5,Valiant,18.1,6,225.0,105
6,Duster 360,14.3,8,360.0,245
7,Merc 240D,24.4,4,146.7,62
8,Merc 230,22.8,4,140.8,95
9,Merc 280,19.2,6,167.6,123


## Updates

In [17]:
# Change data
database = 'mtcars'
sql_query = "UPDATE MTCars SET hp = 111 WHERE name = 'Mazda RX4'"

# Execute
executeQuery(database, sql_query)

# Query data
database = 'mtcars'
sql_query = 'select * from MTCars'

# Return
queryData(database, sql_query)

NameError: name 'executeQuery' is not defined

In [ ]:
# Delete data
database = 'mtcars'
sql_query = "DELETE FROM MTCars WHERE name = 'Mazda RX4'"

# Execute
executeQuery(database, sql_query)

# Query data
database = 'mtcars'
sql_query = 'select * from MTCars'

# Return
queryData(database, sql_query)

___